In [6]:
from langchain.embeddings import OpenAIEmbeddings
from retrievers import VectorStoreRetriever
from config import openai_key
from functions import devrev_functions

openai_embeddings = OpenAIEmbeddings(
    openai_api_key = openai_key
)

vs_retriever = VectorStoreRetriever(
    embeddings=openai_embeddings,
    name = "vs_rtr_01",
    init_functions=devrev_functions.copy()
)

In [5]:
vs_retriever.find_functions('what are my P0 issues')

[Document(page_content='{"name": "resolution_status", "description": "Gives an array of only unresolved issues", "parameters": {"type": "object", "properties": {"unresolved_issue": {"title": "unresolved_issue", "type": "array", "description": "An array of only unresolved issues."}}, "required": []}}', metadata={'index': 16}),
 Document(page_content='{"name": "works_list", "description": "Returns a list of work items matching the request.", "parameters": {"type": "object", "properties": {"applies_to_part": {"title": "applies_to_part", "type": "array of strings", "description": "Filters for work belonging to any of the provided parts."}, "created_by": {"title": "created_by", "type": "array of strings", "description": "Filters for work created by any of these users."}, "issue.priority": {"title": "issue.priority", "type": ["P0", "P1", "P2", "P3"], "description": "Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2, p3."}, "issue.rev_orgs": {"title": "issue.r

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from retrievers import CustomMultiQueryRetriever
from langchain.chat_models import ChatOpenAI
from config import openai_key
from functions import devrev_functions


hf_embeddings = HuggingFaceEmbeddings()

chat_llm = ChatOpenAI(
    openai_api_key = openai_key,
    temperature=0.7,
)

cmq_ret = CustomMultiQueryRetriever(
    chat_llm,
    embeddings=hf_embeddings,
    name = "cmq_ret_01",
    init_functions=devrev_functions.copy()
)

c:\Users\kayde\OneDrive\Desktop\devrev-openai-classic\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
ret_fun = cmq_ret.find_functions('Search for objects related to the customer CustomerABC and create action items from the search results.')

In [6]:
from langchain.chat_models import ChatOpenAI

from verification import Verifier

chat_llm = ChatOpenAI(
    openai_api_key = openai_key
)

verifier = Verifier(
    chat_llm = chat_llm
)

In [24]:
query = "Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1"
schema = '''
[
    {
        "tool_name": "get_similar_work_items",
        "arguments": [
            {
                "argument_name": "work_id",
                "argument_value": "don:core:dvrv-us-1:devo/0:issue/1"
            }
        ]
    },
    {
        "tool_name": "summarize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[0]"
            }
        ]
    }
]'''
verifier(answer=schema,query=query,init_functions=devrev_functions)['text']



'False'

In [13]:
from composers.cot_composer import ChainOfThoughtComposer
from examples import example_queries

example_str = "".join(
    [
        f"""
        Query:
        {query}
        Answer:
        {functions}
        """
        for query, functions in example_queries[:4]
    ]
)

composer = ChainOfThoughtComposer(
    chat_llm=chat_llm,
)

In [15]:
query = 'Search for objects related to the customer CustomerABC and create action items from the search results.'
res = composer(
    query,
    ret_fun,
    example_str
)['text']

In [17]:
print(res)


[
    {
        "tool_name": "search_object_by_name",
        "arguments": [
            {
                "argument_name": "query",
                "argument_value": "CustomerABC"
            }
        ]
    },
    {
        "tool_name": "create_actionable_tasks_from_text",
        "arguments": [
            {
                "argument_name": "text",
                "argument_value": "$$PREV[0]"
            }
        ]
    }
]
